# Covid19 Analyzer
数据分析和展示应包括:

1) 15 天中，全球新冠疫情的总体变化趋势;

2) 15 天中，每日新增确诊数累计排名前 10 个国家的每日新增确诊数据的曲线图;

3) 累计确诊数排名前 10 的国家名称及其数量;

4) 用饼图展示各个国家的累计确诊人数的比例(你爬取的所有国家，数据较小的国家
可以合并处理);

5) 累计确诊人数占国家总人口比例最高的 10 个国家;

6) 疫苗接种情况(至少接种了一针及以上)，请用地图形式展示;

7) 疫苗接种率(累计疫苗接种人数/国家人数)最低的 10 个国家;

8) 全球 GDP 前十名国家的累计确诊人数箱型图，要有平均值;

9) 死亡率最高的 10 个国家;

10) 其它你希望分析和展示的数据。

以上图形应包括完整的坐标、刻度、标签、图例等，如有必要请配上说明文字，对
图中的内容进行解释。

根据以上数据，列出全世界应对新冠疫情最好的 10 个国家，并说明你的理由。

针对全球累计确诊数，利用前 10 天采集到的数据做后 5 天的预测，并与实际数据进行对比。说明你预测的方法，并分析与实际数据的差距和原因。

## imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## 导入数据

In [ ]:
dfs = []
for i in range(5, 12):
    df = pd.read_csv('./dataFetcher/12.' + str(i) + '_data.csv')

    # 保存为float便于计算
    df['confirmed'].astype(float)
    df['confirmed'].astype(float)
    df['deceased'].astype(float)
    df['vaccinated'].astype(float)
    df['population'].astype(float)

    dfs.append(df)
print(f'{len(dfs)} files imported')

## 计算新增数据

In [ ]:
for df in dfs:
    df['new_confirmed'] = 0